In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import qgrid
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./SZA.csv', index_col=0)
rf_df = pd.read_csv('./rf.csv', index_col=0)
rm_df = pd.read_csv('./rm.csv', index_col=0)

In [3]:
df = df.rename(mapper={'Trdmnt': 'Month', 'Mretwd': 'ri'}, axis=1)

In [4]:
rm_df = rm_df.set_index('Month')

In [5]:
rf_df = rf_df[~rf_df.index.duplicated(keep='first')]

In [6]:
df['rf'] = 0
df['rm'] = 0

## Merge Dataframe

First we need to merge the dataframes of stocks, risk-free rage and market return into a single dataframe, making it easier to process later

In [7]:
from datetime import datetime

In [8]:
def alter_df(row):
    date = row['Month']
    date = datetime.strptime(date, '%b-%y')
    date = date.strftime('%Y-%m')
    row['Month'] = date
    row['rf'] = rf_df.loc[date]['rf']/100 ## divide risk-free rate by 100
    row['rm'] = rm_df.loc[date]['Idxrtn']
    return row

df = df.apply(alter_df, axis=1)

In [9]:
df.head()

,Month,ri,rf,rm
Stkcd,,,,
600000,2008-01,-0.128788,0.003386,-0.134478
600000,2008-02,-0.084130,0.003386,0.011720
600000,2008-03,-0.159744,0.003386,-0.189113
600000,2008-04,0.194758,0.003386,0.044477
600000,2008-05,-0.132099,0.003386,-0.087845


In [10]:
Window = 60

In [11]:
Months = np.unique(df['Month'])

In [12]:
Months.shape[0]

132

In [13]:
Stocks = np.unique(df.index)

In [14]:
Stocks.shape

(1469,)

## Drop Data

For those stocks with less than 132 monthly return data, we simply drop them

There were 1469 stocks, and only 295 remains after we dropped stocks without complete monthly return data

In [15]:
for stock in tqdm(Stocks):
    if df.loc[stock].shape[0] != 132 or df.loc[stock].isnull().any()['ri']: ## missing data for some months
        df = df.drop(stock, axis=0)

100%|██████████| 1469/1469 [00:06<00:00, 240.20it/s]


In [16]:
Stocks = np.unique(df.index)

In [17]:
Stocks.shape

(293,)

## Regress Beta

According to CAPM model, we have $$ E(r_i)=r_f+\beta_i(E(r_m)-r_f) $$

To regress beta for each stock across a 5-year window, we regress:
$$ r_{it}-r_{ft}=\alpha_i+\beta_i(r_{mt}-r_{ft})+\epsilon_{it}$$

where $i$ stands for each stock and $t$ stands for each month in this 5-year window

In [18]:
df_beta = pd.DataFrame(columns=Stocks, index=Months)

In [19]:
def RegressCAPM(stk_window):
    y = stk_window['ri']-stk_window['rf']
    X = stk_window['rm']-stk_window['rf']
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    alpha = results.params.get_values()[0]
    beta = results.params.get_values()[1]
    return beta

In [20]:
for i in tqdm(range(Months.shape[0]-60)):
    date = Months[i+60]
    df_rolling = df.loc[np.isin(df['Month'], Months[i:i+60])]
    groupby = df_rolling.groupby('Stkcd')
    current_stks = list(groupby.indices.keys())
    for stk in current_stks:
        stk_window = groupby.get_group(stk)
        ## drop stocks with only one monthly data, (for example, 600786)
        if stk_window.shape[0] == 1: 
            continue
        stk_beta = RegressCAPM(stk_window)
        df_beta.loc[date][stk] = stk_beta

100%|██████████| 72/72 [01:23<00:00,  1.09s/it]


In [ ]:
df_beta.to_csv('beta.csv')

In [ ]:
df.to_csv('merged_monthlyInfo.csv')